# Comparison of HPL traces

In [1]:
import io
import zipfile
import pandas
from plotnine import *
import plotnine
plotnine.options.figure_size = (12, 8)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # removing annoying Pandas warning

def read_csv(archive_name, csv_name, columns=None, filter_func=lambda x: x):
    archive = zipfile.ZipFile(archive_name)
    df= pandas.read_csv(io.BytesIO(filter_func(archive.read(csv_name))), names=columns)
    df.columns = df.columns.str.strip()
    return df

def filter_pajeng(pajeng_output, keep):
    lines = pajeng_output.split(b'\n')
    lines = [l for l in lines if l.startswith(keep.encode())]
    return b'\n'.join(lines)

def _read_paje_state(archive_name, csv_name, has_msg_size=False, replace_func=('MPI', 'MPI')):
    columns=['type', 'rank', 'container', 'start', 'end', 'duration', 'level', 'function']
    if has_msg_size:
        columns.append('tmp')
    df = read_csv(archive_name, csv_name, columns=columns, filter_func=lambda x: filter_pajeng(x, 'State'))
    df['function'] = df['function'].str.replace(*replace_func).str.strip()  # for some reason Simgrid uses PMPI_Wait and not MPI_Wait
    df['msg_size'] = -1
    if has_msg_size:
        df.loc[df['function'].isin(['MPI_Recv', 'MPI_Send', 'MPI_Isend', 'MPI_Irecv']), 'msg_size'] = df['tmp']
        df['msg_size'] = df['msg_size'].astype(int)
    df['rank'] = df['rank'].str.slice(5).astype(int)  # changing 'rank42' into 42
    return df

def read_mpi_state(archive_name, csv_name):
    df = _read_paje_state(archive_name, csv_name, True)
    return df

def read_smpi_state(archive_name, csv_name):
    df = _read_paje_state(archive_name, csv_name, True, ('PMPI', 'MPI'))
    df['rank'] = -df['rank']  # in SMPI, we have 'rank-42' and not 'rank42', so there remains a - that we should remove
    return df

def read_smpi_link(archive_name, csv_name):
    df = read_csv(archive_name, csv_name,
                  columns=['type', 'level', 'container', 'start', 'end', 'duration', 'commType', 'src', 'dst', 'msg_size'],
                  filter_func=lambda x: filter_pajeng(x, 'Link'))
    df['src'] = df['src'].str.slice(6).astype(int)  # changing 'rank42' into 42
    df['dst'] = df['dst'].str.slice(6).astype(int)  # changing 'rank42' into 42
#    df['remote'] = (df['src'] // 32) != (df['dst'] // 32)
    return df

def read_archive(archive_name, mode, exp_id, drop_func=['MPI_Comm_rank', 'MPI_Comm_size', 'MPI_Comm_split', 'MPI_Comm_free', 'MPI_Init', 'MPI_Finalize']):
    assert mode in ('reality', 'simgrid')
    print('# Mode %s, exp %d' % (mode, exp_id))
    blas_trace = read_csv(archive_name, 'trace_blas_%d.csv' % exp_id)
    print('\tBLAS trace: %6d lines' % len(blas_trace))
    blas_trace['start'] = blas_trace.timestamp
    blas_trace['end'] = blas_trace.timestamp + blas_trace.duration
    blas_trace = blas_trace[['file', 'line', 'function', 'start', 'end', 'rank', 'm', 'n', 'k']]
    blas_trace['msg_size'] = -1
    blas_trace['kind'] = 'BLAS'
    if mode == 'reality':
        mpi_trace = read_mpi_state(archive_name, 'trace_mpi_%d.csv' % exp_id)
    else:
        mpi_trace = read_smpi_state(archive_name, 'trace_mpi_%d.csv' % exp_id)
    print('\tMPI  trace: %6d lines' % len(mpi_trace))
    mpi_trace = mpi_trace[['function', 'start', 'end', 'rank', 'msg_size']]
    mpi_trace['kind'] = 'MPI'
    df = pandas.concat([blas_trace, mpi_trace])
    df['mode'] = mode
    df['rank+1'] = df['rank'] + 1
    old_len = len(df)
    df = df[~df['function'].str.strip().isin(drop_func)]
    print('Removed %d events with functions in %s' % (old_len-len(df), ', '.join(drop_func)))
    if mode == 'reality':
        df_links = None
    else:
        df_links = read_smpi_link(archive_name, 'trace_mpi_%d.csv' % exp_id)
    df['duration'] = df['end'] - df['start']
    df['exp_id'] = exp_id
    return df, df_links

reality_file = 'grenoble_2018-12-17_1828852.zip'

i = 0
traces = []
while True:
    try:
        trace_reality, link_reality = read_archive(reality_file, 'reality', i)
    except KeyError:
        break
    traces.append(trace_reality)
    i += 1

traces = pandas.concat(traces)

# Mode reality, exp 0
	BLAS trace: 524470 lines
	MPI  trace: 844024 lines
Removed 524998 events with functions in MPI_Comm_rank, MPI_Comm_size, MPI_Comm_split, MPI_Comm_free, MPI_Init, MPI_Finalize
# Mode reality, exp 1
	BLAS trace: 1418360 lines
	MPI  trace: 1737914 lines
Removed 1418888 events with functions in MPI_Comm_rank, MPI_Comm_size, MPI_Comm_split, MPI_Comm_free, MPI_Init, MPI_Finalize
# Mode reality, exp 2
	BLAS trace: 411720 lines
	MPI  trace: 731274 lines
Removed 412248 events with functions in MPI_Comm_rank, MPI_Comm_size, MPI_Comm_split, MPI_Comm_free, MPI_Init, MPI_Finalize
# Mode reality, exp 3
	BLAS trace: 590292 lines
	MPI  trace: 909846 lines
Removed 590820 events with functions in MPI_Comm_rank, MPI_Comm_size, MPI_Comm_split, MPI_Comm_free, MPI_Init, MPI_Finalize
# Mode reality, exp 4
	BLAS trace: 380834 lines
	MPI  trace: 700388 lines
Removed 381362 events with functions in MPI_Comm_rank, MPI_Comm_size, MPI_Comm_split, MPI_Comm_free, MPI_Init, MPI_Finalize
# Mode 

## HPL result

In [2]:
res = read_csv(reality_file, 'results.csv')
res[['gflops', 'time']]

,gflops,time
0,2253.0,36.99
1,2456.0,33.93
2,2342.0,35.59
3,2247.0,37.09
4,2556.0,32.60
5,2574.0,32.37
6,2078.0,40.10
7,2335.0,35.69
8,2297.0,36.27
9,2451.0,34.00


## Checking the parameters

In [3]:
read_csv(reality_file, 'exp.csv')

,matrix_size,block_size,proc_p,proc_q,pfact,rfact,bcast,depth,swap,mem_align,process_per_node,thread_per_process
0,50000,128,2,4,1,2,2,1,0,8,1,32
1,50000,128,2,4,1,2,2,1,0,8,1,32
2,50000,128,2,4,1,2,2,1,0,8,1,32
3,50000,128,2,4,1,2,2,1,0,8,1,32
4,50000,128,2,4,1,2,2,1,0,8,1,32
5,50000,128,2,4,1,2,2,1,0,8,1,32
6,50000,128,2,4,1,2,2,1,0,8,1,32
7,50000,128,2,4,1,2,2,1,0,8,1,32
8,50000,128,2,4,1,2,2,1,0,8,1,32
9,50000,128,2,4,1,2,2,1,0,8,1,32


In [4]:
for exp_id in sorted(traces['exp_id'].unique()):
    for kind in trace_reality['kind'].unique():
        df = traces[(traces['exp_id'] == exp_id) & (traces['kind'] == kind)]
        print('exp %d' % exp_id, kind, df.start.min(), df.end.max())

exp 0 BLAS 7.68392 43.932711999999995
exp 0 MPI 2.7000000000000002e-05 54.674389
exp 1 BLAS 7.016 40.891414999999995
exp 1 MPI 4.9e-05 51.533667
exp 2 BLAS 6.992000000000001 42.524018999999996
exp 2 MPI 4.6e-05 52.185956
exp 3 BLAS 7.00425 44.0451
exp 3 MPI 2.7000000000000002e-05 53.769845999999994
exp 4 BLAS 6.98716 39.5477
exp 4 MPI 2.6000000000000002e-05 49.171784
exp 5 BLAS 6.98802 39.323521
exp 5 MPI 2.5e-05 49.025956
exp 6 BLAS 6.992369999999999 47.0485
exp 6 MPI 2.6000000000000002e-05 57.97732
exp 7 BLAS 6.98387 42.633218
exp 7 MPI 3.3e-05 52.333627
exp 8 BLAS 6.9811 43.201108999999995
exp 8 MPI 2.6000000000000002e-05 52.928377000000005
exp 9 BLAS 7.044810000000001 41.005616
exp 9 MPI 2.5e-05 50.705438


## Comparing dgemm traces

In [5]:
def dump_trace(df, function, columns, rank=None):
    columns = columns + ['rank']
    for exp_id in df['exp_id'].unique():
        tmp = df[(df['function'] == function) & (df['exp_id'] == exp_id)][columns]
        if rank is not None:
            tmp = tmp[tmp['rank'] == rank]
        else:
            tmp = tmp.sort_values(by=['rank'])
        filename = '/tmp/trace_%d' % exp_id
        tmp.to_csv(filename, index=False)
        print('Created file %s' % filename)

In [6]:
dgemm = traces[traces['function'] == 'dgemm'].sort_values(by=['start'])
dgemm['size_product'] = dgemm.m * dgemm.n * dgemm.k
group = dgemm.groupby(['exp_id'])[['size_product', 'm', 'n', 'k', 'duration']].agg(['sum'])
group['count'] = dgemm.groupby(['exp_id']).count()['m']
group

,size_product,m,n,k,duration,count
,sum,sum,sum,sum,sum,
exp_id,,,,,,
0,4.158680e+13,2.019803e+09,19781982.0,21566080.0,112.843692,262235
1,4.158680e+13,1.424623e+10,19781982.0,78775040.0,114.916742,709180
2,4.158680e+13,1.774795e+09,19781982.0,14350080.0,115.569613,205860
3,4.158680e+13,3.138485e+09,19781982.0,25778688.0,114.840904,295146
4,4.158680e+13,1.580385e+09,19781982.0,12373376.0,114.126110,190417
5,4.158680e+13,2.288898e+09,19781982.0,24901632.0,114.889643,288294
6,4.158680e+13,2.155638e+09,19781982.0,21817856.0,114.001533,264202
7,4.158680e+13,1.989548e+09,19781982.0,17292928.0,115.879240,228851


In [7]:
dgemm.groupby(['rank', 'exp_id'])[['size_product']].agg(['count', 'sum'])

size_product              
                   count           sum
rank exp_id                           
0    0             21465  5.198147e+12
     1            292019  5.198147e+12
     2             23291  5.198147e+12
     3             45658  5.198147e+12
     4             21616  5.198147e+12
     5             19604  5.198147e+12
     6             23420  5.198147e+12
     7             24736  5.198147e+12
     8             22793  5.198147e+12
     9             34048  5.198147e+12
1    0             46607  5.218160e+12
     1             22060  5.218160e+12
     2             22208  5.218160e+12
     3             21915  5.218160e+12
     4             20850  5.218160e+12
     5             49079  5.218160e+12
     6             23239  5.218160e+12
     7             20510  5.218160e+12
     8             20177  5.218160e+12
     9             22467  5.218160e+12
2    0             38304  5.208855e+12
     1             29085  5.208855e+12
     2             23448  5.208855e+12
     3             46545  5.208855e+12
     4             32747  5.208855e+12
     5             46321  5.208855e+12
     6             64665  5.208855e+12
     7             42978  5.208855e+12
     8            164609  5.208855e+12
     9             27994  5.208855e+12
...                  ...           ...
5    0             53831  5.213241e+12
     1             24122  5.213241e+12
     2             22604  5.213241e+12
     3             21702  5.213241e+12
     4             22024  5.213241e+12
     5             52713  5.213241e+12
     6             20840  5.213241e+12
     7             31928  5.213241e+12
     8             30161  5.213241e+12
     9             22446  5.213241e+12
6    0             34576  5.203845e+12
     1             35603  5.203845e+12
     2             23157  5.203845e+12
     3             37342  5.203845e+12
     4             23150  5.203845e+12
     5             52870  5.203845e+12
     6             52858  5.203845e+12
     7             27209  5.203845e+12
     8            133706  5.203845e+12
     9             26529  5.203845e+12
7    0             23488  5.173267e+12
     1             24005  5.173267e+12
     2             37921  5.173267e+12
     3             22697  5.173267e+12
     4             24709  5.173267e+12
     5             24066  5.173267e+12
     6             24271  5.173267e+12
     7             22083  5.173267e+12
     8             21818  5.173267e+12
     9             28814  5.173267e+12

[80 rows x 2 columns]

## Comparing dtrsm traces

In [8]:
dtrsm = traces[traces['function'] == 'dtrsm'].sort_values(by=['start'])
dtrsm['size_product'] = dtrsm.m * dtrsm.n ** 2
group = dtrsm.groupby(['exp_id'])[['size_product', 'm', 'n', 'duration']].agg(['sum'])
group['count'] = dtrsm.groupby(['exp_id']).count()['m']
group

,size_product,m,n,duration,count
,sum,sum,sum,sum,
exp_id,,,,,
0,3.194668e+11,19781982.0,21566080.0,2.842374,262235
1,3.194668e+11,19781982.0,78775040.0,2.836802,709180
2,3.194668e+11,19781982.0,14350080.0,3.297929,205860
3,3.194668e+11,19781982.0,25778688.0,3.049582,295146
4,3.194668e+11,19781982.0,12373376.0,2.845975,190417
5,3.194668e+11,19781982.0,24901632.0,2.870475,288294
6,3.194668e+11,19781982.0,21817856.0,3.026892,264202
7,3.194668e+11,19781982.0,17292928.0,3.074233,228851


In [9]:
dtrsm.groupby(['rank', 'exp_id'])[['size_product']].agg(['count', 'sum'])

size_product              
                   count           sum
rank exp_id                           
0    0             21465  3.990530e+10
     1            292019  3.990530e+10
     2             23291  3.990530e+10
     3             45658  3.990530e+10
     4             21616  3.990530e+10
     5             19604  3.990530e+10
     6             23420  3.990530e+10
     7             24736  3.990530e+10
     8             22793  3.990530e+10
     9             34048  3.990530e+10
1    0             46607  4.011082e+10
     1             22060  4.011082e+10
     2             22208  4.011082e+10
     3             21915  4.011082e+10
     4             20850  4.011082e+10
     5             49079  4.011082e+10
     6             23239  4.011082e+10
     7             20510  4.011082e+10
     8             20177  4.011082e+10
     9             22467  4.011082e+10
2    0             38304  4.001576e+10
     1             29085  4.001576e+10
     2             23448  4.001576e+10
     3             46545  4.001576e+10
     4             32747  4.001576e+10
     5             46321  4.001576e+10
     6             64665  4.001576e+10
     7             42978  4.001576e+10
     8            164609  4.001576e+10
     9             27994  4.001576e+10
...                  ...           ...
5    0             53831  4.011082e+10
     1             24122  4.011082e+10
     2             22604  4.011082e+10
     3             21702  4.011082e+10
     4             22024  4.011082e+10
     5             52713  4.011082e+10
     6             20840  4.011082e+10
     7             31928  4.011082e+10
     8             30161  4.011082e+10
     9             22446  4.011082e+10
6    0             34576  4.001576e+10
     1             35603  4.001576e+10
     2             23157  4.001576e+10
     3             37342  4.001576e+10
     4             23150  4.001576e+10
     5             52870  4.001576e+10
     6             52858  4.001576e+10
     7             27209  4.001576e+10
     8            133706  4.001576e+10
     9             26529  4.001576e+10
7    0             23488  3.970153e+10
     1             24005  3.970153e+10
     2             37921  3.970153e+10
     3             22697  3.970153e+10
     4             24709  3.970153e+10
     5             24066  3.970153e+10
     6             24271  3.970153e+10
     7             22083  3.970153e+10
     8             21818  3.970153e+10
     9             28814  3.970153e+10

[80 rows x 2 columns]

## Comparing MPI_Recv traces

In [10]:
recv = traces[traces['function'] == 'MPI_Recv'].sort_values(by=['start'])
group = recv.groupby(['exp_id'])[['msg_size', 'duration']].agg(['sum'])
group['count'] = recv.groupby(['exp_id']).count()['start']
group

,msg_size,duration,count
,sum,sum,
exp_id,,,
0,30238677876,23.409467,3682
1,30238677876,18.689834,3682
2,30238677876,15.270480,3682
3,30238677876,16.679846,3682
4,30238677876,12.937636,3682
5,30238677876,12.688172,3682
6,30238677876,25.928855,3682
7,30238677876,15.732654,3682


In [11]:
recv.groupby(['rank', 'exp_id'])[['msg_size']].agg(['count', 'sum'])

msg_size            
               count         sum
rank exp_id                     
0    0           421  3776632664
     1           421  3776632664
     2           421  3776632664
     3           421  3776632664
     4           421  3776632664
     5           421  3776632664
     6           421  3776632664
     7           421  3776632664
     8           421  3776632664
     9           421  3776632664
1    0           505  3775431788
     1           505  3775431788
     2           505  3775431788
     3           505  3775431788
     4           505  3775431788
     5           505  3775431788
     6           505  3775431788
     7           505  3775431788
     8           505  3775431788
     9           505  3775431788
2    0           416  3788906148
     1           416  3788906148
     2           416  3788906148
     3           416  3788906148
     4           416  3788906148
     5           416  3788906148
     6           416  3788906148
     7           416  3788906148
     8           416  3788906148
     9           416  3788906148
...              ...         ...
5    0           405  3776868460
     1           405  3776868460
     2           405  3776868460
     3           405  3776868460
     4           405  3776868460
     5           405  3776868460
     6           405  3776868460
     7           405  3776868460
     8           405  3776868460
     9           405  3776868460
6    0           510  3777548044
     1           510  3777548044
     2           510  3777548044
     3           510  3777548044
     4           510  3777548044
     5           510  3777548044
     6           510  3777548044
     7           510  3777548044
     8           510  3777548044
     9           510  3777548044
7    0           405  3789713524
     1           405  3789713524
     2           405  3789713524
     3           405  3789713524
     4           405  3789713524
     5           405  3789713524
     6           405  3789713524
     7           405  3789713524
     8           405  3789713524
     9           405  3789713524

[80 rows x 2 columns]

In [12]:
dump_trace(traces, 'MPI_Recv', ['msg_size'])

Created file /tmp/trace_0
Created file /tmp/trace_1
Created file /tmp/trace_2
Created file /tmp/trace_3
Created file /tmp/trace_4
Created file /tmp/trace_5
Created file /tmp/trace_6
Created file /tmp/trace_7
Created file /tmp/trace_8
Created file /tmp/trace_9


In [13]:
!head /tmp/trace_0
!wc -l /tmp/trace_*
!sha256sum /tmp/trace_*

msg_size,rank
8,0
14894088,0
14763016,0
14763016,0
14631944,0
14500872,0
14500872,0
14369800,0
14238728,0
  3683 /tmp/trace_0
  3683 /tmp/trace_1
  3683 /tmp/trace_2
  3683 /tmp/trace_3
  3683 /tmp/trace_4
  3683 /tmp/trace_5
  3683 /tmp/trace_6
  3683 /tmp/trace_7
  3683 /tmp/trace_8
  3683 /tmp/trace_9
 36830 total
62fe773b68fa098bc959c9c322b997c12a57936163c9775e3c5e7befbcd58a1a  /tmp/trace_0
62fe773b68fa098bc959c9c322b997c12a57936163c9775e3c5e7befbcd58a1a  /tmp/trace_1
62fe773b68fa098bc959c9c322b997c12a57936163c9775e3c5e7befbcd58a1a  /tmp/trace_2
62fe773b68fa098bc959c9c322b997c12a57936163c9775e3c5e7befbcd58a1a  /tmp/trace_3
62fe773b68fa098bc959c9c322b997c12a57936163c9775e3c5e7befbcd58a1a  /tmp/trace_4
62fe773b68fa098bc959c9c322b997c12a57936163c9775e3c5e7befbcd58a1a  /tmp/trace_5
62fe773b68fa098bc959c9c322b997c12a57936163c9775e3c5e7befbcd58a1a  /tmp/trace_6
62fe773b68fa098bc959c9c322b997c12a57936163c9775e3c5e7befbcd58a1a  /tmp/trace_7
62fe773b68fa098bc959c9c322b997c12a57936163c9775e3

## Comparing MPI_Send traces

In [14]:
send = traces[traces['function'] == 'MPI_Send'].sort_values(by=['start'])
group = send.groupby(['exp_id'])[['msg_size', 'duration']].agg(['sum'])
group['count'] = send.groupby(['exp_id']).count()['start']
group

,msg_size,duration,count
,sum,sum,
exp_id,,,
0,45357274452,86.872126,107564
1,45357274452,72.639166,107564
2,45357274452,81.284955,107564
3,45357274452,90.207162,107564
4,45357274452,66.347361,107564
5,45357274452,65.085441,107564
6,45357274452,106.897887,107564
7,45357274452,81.733353,107564


In [15]:
send.groupby(['rank', 'exp_id'])[['msg_size']].agg(['count', 'sum'])

msg_size            
               count         sum
rank exp_id                     
0    0         13557  5684327908
     1         13557  5684327908
     2         13557  5684327908
     3         13557  5684327908
     4         13557  5684327908
     5         13557  5684327908
     6         13557  5684327908
     7         13557  5684327908
     8         13557  5684327908
     9         13557  5684327908
1    0         13431  5684827848
     1         13431  5684827848
     2         13431  5684827848
     3         13431  5684827848
     4         13431  5684827848
     5         13431  5684827848
     6         13431  5684827848
     7         13431  5684827848
     8         13431  5684827848
     9         13431  5684827848
2    0         13501  5677111900
     1         13501  5677111900
     2         13501  5677111900
     3         13501  5677111900
     4         13501  5677111900
     5         13501  5677111900
     6         13501  5677111900
     7         13501  5677111900
     8         13501  5677111900
     9         13501  5677111900
...              ...         ...
5    0         13527  5665138520
     1         13527  5665138520
     2         13527  5665138520
     3         13527  5665138520
     4         13527  5665138520
     5         13527  5665138520
     6         13527  5665138520
     7         13527  5665138520
     8         13527  5665138520
     9         13527  5665138520
6    0         13401  5676510180
     1         13401  5676510180
     2         13401  5676510180
     3         13401  5676510180
     4         13401  5676510180
     5         13401  5676510180
     6         13401  5676510180
     7         13401  5676510180
     8         13401  5676510180
     9         13401  5676510180
7    0         13394  5632961528
     1         13394  5632961528
     2         13394  5632961528
     3         13394  5632961528
     4         13394  5632961528
     5         13394  5632961528
     6         13394  5632961528
     7         13394  5632961528
     8         13394  5632961528
     9         13394  5632961528

[80 rows x 2 columns]

In [16]:
dump_trace(traces, 'MPI_Send', ['msg_size'])

Created file /tmp/trace_0
Created file /tmp/trace_1
Created file /tmp/trace_2
Created file /tmp/trace_3
Created file /tmp/trace_4
Created file /tmp/trace_5
Created file /tmp/trace_6
Created file /tmp/trace_7
Created file /tmp/trace_8
Created file /tmp/trace_9


In [17]:
!head /tmp/trace_0
!wc -l /tmp/trace_*
!sha256sum /tmp/trace_*

msg_size,rank
8,0
2080,0
2080,0
2080,0
2080,0
2080,0
2080,0
2080,0
2080,0
 107565 /tmp/trace_0
 107565 /tmp/trace_1
 107565 /tmp/trace_2
 107565 /tmp/trace_3
 107565 /tmp/trace_4
 107565 /tmp/trace_5
 107565 /tmp/trace_6
 107565 /tmp/trace_7
 107565 /tmp/trace_8
 107565 /tmp/trace_9
1075650 total
cce3738f61a97849b79b9d08eb241eca3f469f4e7b1565b9e66f70652ab6b912  /tmp/trace_0
8d82b1d8d6219efedebd21648c819014e0edfe9b6dccde21428e0f18414f105e  /tmp/trace_1
f6d66e4b2708f078290dc245c167ddf73f8e0e784239aaea25ca65b511eb92d6  /tmp/trace_2
3f5840bc0b49af4a668809ef45ffc7e340a3891f7ed581ec81f7e46bc32e6a79  /tmp/trace_3
66f2827c328e647ee53093d968439f253c04c63dc9038c22bb7a6bf5eb3ba7b4  /tmp/trace_4
1dd19b3a18e6dcbef4ec6b87f76ab57fe811a12551c1aa5a2d622cf65cd7a0bf  /tmp/trace_5
bae15c60ecc43e022b704372aff22459843d3cdaf46fe8152081e73433037505  /tmp/trace_6
183493409d00b9dbbcb69056879510e04ba408ff4c4ad6e07495c2ee2265992a  /tmp/trace_7
b35b8e90958010d3e676bc517a0877b3de98deb878030e77e877710c341724aa  /tmp

In [18]:
!git diff --no-index /tmp/trace_0 /tmp/trace_1

diff --git a/tmp/trace_0 b/tmp/trace_1
index 39bcd4a..ed5f85e 100644
--- a/tmp/trace_0
+++ b/tmp/trace_1
@@ -145,8 +145,8 @@ msg_size,rank
 2080,0
 63984,0
 8388608,0
-16860168,0
 3998184,0
+16860168,0
 8388608,0
 3932640,0
 16991240,0
@@ -509,8 +509,8 @@ msg_size,rank
 2080,0
 58824,0
 8126464,0
-3936752,0
 16335880,0
+3936752,0
 8126464,0
 4127240,0
 16466952,0
@@ -1181,8 +1181,8 @@ msg_size,rank
 2080,0
 61920,0
 9437184,0
-5235256,0
 18957320,0
+5235256,0
 9437184,0
 5751408,0
 19088392,0
@@ -1875,8 +1875,8 @@ msg_size,rank
 2080,0
 2080,0
 2080,0
-13714440,0
 3781176,0
+13714440,0
 6815744,0
 2080,0
 2080,0
@@ -2289,8 +2289,8 @@ msg_size,rank
 2080,0
 2080,0
 6422528,0
-12928008,0
 2860488,0
+12928008,0
 6422528,0
 3412512,0
 13059080,0
@@ -3030,8 +3030,8 @@ msg_size,rank
 2080,0
 2080,0
 70176,0
-3842584,0
 14763016,0
+3842584,0
 7340032,0
 3842584,0
 14894088,0
@@ -3504,7 +3504,7 @@ msg_size,rank
 2080,0
 2080,0
 2080,0
-6288936,0
+23413768,0
 2080,0
 2080,0
 2080,0
@@ -3517,7 +

## Comparing MPI_Irecv traces

In [19]:
irecv = traces[traces['function'] == 'MPI_Irecv'].sort_values(by=['start'])
group = irecv.groupby(['exp_id'])[['msg_size', 'duration']].agg(['sum'])
group['count'] = recv.groupby(['exp_id']).count()['start']
group

,msg_size,duration,count
,sum,sum,
exp_id,,,
0,15118596576,1.857949,3682
1,15118596576,1.313817,3682
2,15118596576,1.664459,3682
3,15118596576,1.674129,3682
4,15118596576,1.331773,3682
5,15118596576,1.331842,3682
6,15118596576,1.891051,3682
7,15118596576,1.507120,3682


In [20]:
irecv.groupby(['rank', 'exp_id'])[['msg_size']].agg(['count', 'sum'])

msg_size            
               count         sum
rank exp_id                     
0    0         13028  1880993888
     1         13028  1880993888
     2         13028  1880993888
     3         13028  1880993888
     4         13028  1880993888
     5         13028  1880993888
     6         13028  1880993888
     7         13028  1880993888
     8         13028  1880993888
     9         13028  1880993888
1    0         13030  1900235280
     1         13030  1900235280
     2         13030  1900235280
     3         13030  1900235280
     4         13030  1900235280
     5         13030  1900235280
     6         13030  1900235280
     7         13030  1900235280
     8         13030  1900235280
     9         13030  1900235280
2    0         12984  1886076864
     1         12984  1886076864
     2         12984  1886076864
     3         12984  1886076864
     4         12984  1886076864
     5         12984  1886076864
     6         12984  1886076864
     7         12984  1886076864
     8         12984  1886076864
     9         12984  1886076864
...              ...         ...
5    0         13030  1896120808
     1         13030  1896120808
     2         13030  1896120808
     3         13030  1896120808
     4         13030  1896120808
     5         13030  1896120808
     6         13030  1896120808
     7         13030  1896120808
     8         13030  1896120808
     9         13030  1896120808
6    0         12984  1901227680
     1         12984  1901227680
     2         12984  1901227680
     3         12984  1901227680
     4         12984  1901227680
     5         12984  1901227680
     6         12984  1901227680
     7         12984  1901227680
     8         12984  1901227680
     9         12984  1901227680
7    0         12899  1876865144
     1         12899  1876865144
     2         12899  1876865144
     3         12899  1876865144
     4         12899  1876865144
     5         12899  1876865144
     6         12899  1876865144
     7         12899  1876865144
     8         12899  1876865144
     9         12899  1876865144

[80 rows x 2 columns]

In [21]:
dump_trace(traces, 'MPI_Irecv', ['msg_size'])

Created file /tmp/trace_0
Created file /tmp/trace_1
Created file /tmp/trace_2
Created file /tmp/trace_3
Created file /tmp/trace_4
Created file /tmp/trace_5
Created file /tmp/trace_6
Created file /tmp/trace_7
Created file /tmp/trace_8
Created file /tmp/trace_9


In [22]:
!head /tmp/trace_0
!wc -l /tmp/trace_*
!sha256sum /tmp/trace_*

msg_size,rank
1056,0
1056,0
1056,0
1056,0
1056,0
1056,0
1056,0
1056,0
1056,0
 103883 /tmp/trace_0
 103883 /tmp/trace_1
 103883 /tmp/trace_2
 103883 /tmp/trace_3
 103883 /tmp/trace_4
 103883 /tmp/trace_5
 103883 /tmp/trace_6
 103883 /tmp/trace_7
 103883 /tmp/trace_8
 103883 /tmp/trace_9
1038830 total
e751a8721e7470781cfbd8dc74d7dd9d9d0da7dbcd2a5ab4138918bdfcdd3520  /tmp/trace_0
e751a8721e7470781cfbd8dc74d7dd9d9d0da7dbcd2a5ab4138918bdfcdd3520  /tmp/trace_1
e751a8721e7470781cfbd8dc74d7dd9d9d0da7dbcd2a5ab4138918bdfcdd3520  /tmp/trace_2
e751a8721e7470781cfbd8dc74d7dd9d9d0da7dbcd2a5ab4138918bdfcdd3520  /tmp/trace_3
e751a8721e7470781cfbd8dc74d7dd9d9d0da7dbcd2a5ab4138918bdfcdd3520  /tmp/trace_4
e751a8721e7470781cfbd8dc74d7dd9d9d0da7dbcd2a5ab4138918bdfcdd3520  /tmp/trace_5
e751a8721e7470781cfbd8dc74d7dd9d9d0da7dbcd2a5ab4138918bdfcdd3520  /tmp/trace_6
e751a8721e7470781cfbd8dc74d7dd9d9d0da7dbcd2a5ab4138918bdfcdd3520  /tmp/trace_7
e751a8721e7470781cfbd8dc74d7dd9d9d0da7dbcd2a5ab4138918bdfcdd3520  /

## In-depth look at dgemm

In [23]:
dgemm.head()

,end,file,function,k,kind,line,m,msg_size,n,rank,start,mode,rank+1,duration,exp_id,size_product
1,6.981264,../HPL_pdrpanrlT.c,dgemm,2.0,BLAS,224.0,25038.0,-1,2.0,0,6.98115,reality,1,0.000114,8,100152.0
469979,6.981296,../HPL_pdrpanrlT.c,dgemm,2.0,BLAS,224.0,24960.0,-1,2.0,4,6.98121,reality,5,0.000086,8,99840.0
3,6.981441,../HPL_pdrpanrlT.c,dgemm,2.0,BLAS,224.0,25036.0,-1,0.0,0,6.98144,reality,1,0.000001,8,0.0
5,6.981707,../HPL_pdrpanrlT.c,dgemm,4.0,BLAS,224.0,25036.0,-1,4.0,0,6.98145,reality,1,0.000257,8,400576.0
469981,6.981520,../HPL_pdrpanrlT.c,dgemm,2.0,BLAS,224.0,24960.0,-1,0.0,4,6.98152,reality,5,0.000000,8,0.0


In [24]:
dgemm['line'] = dgemm['line'].astype(int)
dgemm[['file', 'line']].drop_duplicates().reset_index(drop=True)

,file,line
0,../HPL_pdrpanrlT.c,224
1,../HPL_pdupdateTT.c,384
2,../HPL_pdupdateTT.c,407
3,../HPL_pdupdateTT.c,326
4,../HPL_pdupdateTT.c,349


In [25]:
group = dgemm.groupby(['file', 'line', 'exp_id'])[['duration']].agg(['count', 'sum'])
group

duration           
                                    count        sum
file                 line exp_id                    
 ../HPL_pdrpanrlT.c  224  0         98436   3.589812
                          1         98436   3.642690
                          2         98436   3.637559
                          3         98436   3.616940
                          4         98436   3.659490
                          5         98436   3.613666
                          6         98436   3.613431
                          7         98436   3.627405
                          8         98436   3.646484
                          9         98436   3.630002
 ../HPL_pdupdateTT.c 326  0         88827   5.664168
                          1        295116   6.654223
                          2         51728   9.459513
                          3         82152   5.834199
                          4         52394   6.210165
                          5        112248   6.793103
                          6        104437   7.227697
                          7         65741   7.640609
                          8        154229  10.692205
                          9         56979   7.861471
                     349  0         71332   5.395777
                          1        311966   4.617241
                          2         52031   7.381091
                          3        110922   7.921720
                          4         35900   4.246097
                          5         74004   4.520566
                          6         57678   6.518549
                          7         61035   7.016326
                          8        193159   7.518298
                          9         53816   6.936870
                     384  0          1812  48.992546
                          1          1808  49.226228
                          2          1833  46.725157
                          3          1822  49.406478
                          4          1829  49.246844
                          5          1787  49.165328
                          6          1807  47.804135
                          7          1807  48.367715
                          8          1771  46.046929
                          9          1835  48.606056
                     407  0          1828  49.201389
                          1          1854  50.776360
                          2          1832  48.366293
                          3          1814  48.061567
                          4          1858  50.763514
                          5          1819  50.796980
                          6          1844  48.837721
                          7          1832  49.227185
                          8          1782  48.637860
                          9          1848  49.772292

In [26]:
tmp = dgemm[dgemm.size_product > 0]
group = tmp.groupby(['file', 'line', 'exp_id'])[['duration']].agg(['count', 'sum'])
group

duration           
                                    count        sum
file                 line exp_id                    
 ../HPL_pdrpanrlT.c  224  0         49179   3.586053
                          1         49179   3.638884
                          2         49179   3.633606
                          3         49179   3.613158
                          4         49179   3.655811
                          5         49179   3.610030
                          6         49179   3.609446
                          7         49179   3.623509
                          8         49179   3.642767
                          9         49179   3.626359
 ../HPL_pdupdateTT.c 326  0          6906   5.661168
                          1          7592   6.643169
                          2         10064   9.457667
                          3          6690   5.831098
                          4          7308   6.208353
                          5          8650   6.789104
                          6          8335   7.224080
                          7          8847   7.638207
                          8         12013  10.686907
                          9          8501   7.859600
                     349  0          6408   5.393293
                          1          5203   4.602853
                          2          7993   7.379275
                          3          8678   7.917380
                          4          4831   4.244887
                          5          5649   4.517906
                          6          7069   6.516617
                          7          7821   7.014295
                          8          8338   7.511346
                          9          7372   6.934967
                     384  0          1809  48.992546
                          1          1805  49.226228
                          2          1830  46.725156
                          3          1819  49.406478
                          4          1826  49.246844
                          5          1784  49.165328
                          6          1804  47.804135
                          7          1804  48.367715
                          8          1768  46.046929
                          9          1832  48.606056
                     407  0          1827  49.201389
                          1          1853  50.776360
                          2          1831  48.366293
                          3          1813  48.061567
                          4          1857  50.763514
                          5          1818  50.796980
                          6          1843  48.837721
                          7          1831  49.227185
                          8          1781  48.637860
                          9          1847  49.772292